**Étape 1 : Obtention du summonerId des joueurs classés diamant dans le mode compétitif de League of Legends**

L'"api_key" est la clé permettant d'utiliser l'API de Riot, le développeur de League of Legends.
Cet "api_key" n'est valide que 24h et permet uniquement de réaliser 100 requêtes toutes les 2 minutes.

L'API de LoL est disponible sur le lien suivant : https://developer.riotgames.com/ . Cependant pour pouvoir l'utiliser il faut s'identifier sur la plateforme, suite à quoi on pourra récupérer notre "api_key".

La variable "region" correspond à la région depuis laquelle on souhaite récupérer les données, ici l'europe. Selon les requêtes que l'on demande à l'API, region sera soit "euw1", sois "europe" comme on le verra plus tard. Ce changement de nom de variable est dû au fait que Riot met régulièrement à jour la structure de son API.

In [1]:
import requests
import time
import pandas as pd
import numpy as np

api_key = "RGAPI-f1b15f32-267d-4ccc-b36b-4fce4d438af6"
region = "euw1"

L'API de LoL nous permet de récupérer le summonerId des joueurs d'un certain rang (ici on s'intéresse aux joueurs de rang diamant) : https://developer.riotgames.com/apis#league-exp-v4/GET_getLeagueEntries . On retire ensuite le lien "url_pull" qui se trouve dans le programme suivant.

Les données sur l'API sont organisées de la manière suivante : Par Palier (IRON, BRONZE, SILVER, ... , DIAMANT, ...) ; Puis pour chaque palier, les joueurs sont divisés dans des Tiers ( I , II , III , IV ) ; Et enfin on accède aux summonerId sur des pages avec 205 ID/page.

De notre côté nous allons récupérer les summonerId de 20 pages dans chaque tier du palier diamant, sois 16400 identifiants.

In [3]:
def summ_ID_puller(division, tier, page):

    url_pull = "https://{}.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{}/{}?page={}&api_key={}".format(
        region, division, tier, page, api_key)
    profile_list = requests.get(url_pull).json()
    summID_list = []

    for profile in range(len(profile_list)):
        summID_list.append(profile_list[profile]['summonerId'])

    df = pd.DataFrame(summID_list, columns=["Summoner ID"])
    df.to_csv('summID.csv', mode='a')

Comme dit plus tôt, on ne peut réaliser que 100 requêtes toutes les 2 minutes, on applique donc un temps de pause entre chaque ittération de notre boucle.

In [7]:
for tier in ["I", "II", "III", "IV"]:
    for page in range(1, 20):
        time.sleep(1.25)
        summ_ID_puller("DIAMOND", tier, page)

Nous avons maintenant une liste des summonerId de nombreux joueurs, on va maintenant chercher à obtenir leur historique pour avoir une base de données de parties jouées, base sur laquelle on va travailler au cours de ce projet.

Cepdandant, pour accéder à l'historique des parties, l'API de Riot requière un autre identifiant unique aux joueurs, le PUUID, ce que nous allons à présent récupérer.

**Étape 2 : Convertir les summonerId en PUUID**

In [5]:
import requests
import time
import pandas as pd
import numpy as np

summoner_IDs = pd.read_csv("summID.csv")
accountID_list = []
summID_list = summoner_IDs["Summoner PUUID"]

On utilise encore simplement l'API de Riot, elle est si pratique, pourquoi s'en priverait-on !

In [6]:
def acct_ID_puller(summID):
    url_acct_pull = "https://{}.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}".format(
        region, summID, api_key)
    account_info = requests.get(url_acct_pull).json()
    accountID_list.append(account_info["puuid"])

Ici on remarque que je ne récupère que les PUUID des 100 premières personnes de ma liste, en effet, lorsque nous allons chercher à récupérer l'historique de nos joueurs plus tard, nous allons voir que le nombre de requêtes va rapidement augmenter jusqu'à plusieurs heures, et étant donné qu'avec même juste une centaine de PUUID, on n'a accès à plus de 7000 parties sur les mois d'octobre et novembre, nous avons décider de nous contenter de cela pour l'instant.

In [6]:
for summID_idx in range(0, 100):
    time.sleep(1.5)
    if summID_list[summID_idx] == "Summoner ID":
        pass
    else:
        try:
            acct_ID_puller(summID_list[summID_idx])
        except KeyError:
            print("keyerror")

KeyboardInterrupt: 

In [7]:
df = pd.DataFrame(accountID_list, columns=["AccountPUUID"])
df.to_csv('accountPUUID.csv', mode='x')

FileExistsError: [Errno 17] File exists: 'accountPUUID.csv'

**Étape 3 : Récupération de l'ID de matchs"**

A présent nous allons enfin commencer à nous créer une base de données de parties. Dans un premier temps, pour chaque joueur de notre base, nous allons récupérer son historique de partie sous la forme d'une liste d'ID avec laquelle on pourra dans un second temps obtenir toutes les données de parties.

In [12]:
import requests
import time
import pandas as pd
import numpy as np

region1 = "europe"
account_IDs = pd.read_csv("accountPUUID.csv")
account_IDs_list = account_IDs["AccountPUUID"]

Au début, nous voulions récupérer les données de toute l'année, cependant nous nous sommes rendus compte plus tard que cela posait un problème dans notre modèle, c'est pourquoi dans la suite nous nous contenterons des données des 2 derniers mois seulement.

*Pour expliquer dans les grandes lignes le problème, certaines données utile à notre modèle nous demandent de réaliser un très grand nombre de requête via l'API (donc trop long), en plus d'être relativement coûteux en complexité. De plus le programme à écrire pour récupérer les données, bien que relativement simple dans le principe, est très embêtant à écrire à cause du grand nombre de variables, nous nous en sommes donc dispensés ^^. A la place nous avons préféré utiliser des bases de données déjà disponible en ligne sur des sites que l'on récupère par scraping. Mais voiçi donc le probrème : les données sur les joueurs sont actualisées à la date d'aujourd'hui et ne correspondent donc plus aux statistiques datant du moment où les parties ont été joué. Pour éviter de créer des incertitudes trop importantes, on s'est concentré uniquement sur les parties des deux derniers mois.*

In [45]:
#La liste des dates : date exacte des débuts et fins de mois en format "Timestamp"
dates = np.array([["Janvier", [1640995200,1643673599]],["Février", [1643673600,1646092799]],["Mars", [1646092800,1648771199]],
         ["Avril", [1648771200,1651363199]],["Mai", [1651363200,1654041599]],["Juin", [1654041600,1656633599]],
         ["Juillet", [1656633600,1659311999]],["Aout", [1659312000,1661990399]],["Septembre", [1661990400,1664582399]],
         ["Octobre", [1664582400,1667260799]],["Novembre", [1667174400,1669852799]]], dtype=object)

#On définit en amont le DataFrame de travail et le DataFrame final
df_vide = pd.DataFrame(columns=[i[0] for i in dates])
df1 = df_vide.copy()

Le principe est le même qu'à l'étape précédente.

In [10]:
def match_ID_puller(matchID_list, puuid, startTime,endTime):
    url_match_pull = "https://{}.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?startTime={}&endTime={}&queue=420" \
                     "&type=ranked&start=0&count=100&api_key={}".format(region1, puuid, startTime, endTime, api_key)
    match_history = requests.get(url_match_pull).json()
    matchID_list += match_history

In [11]:
for mois in dates:
    
    matchID_list = []
    df_travail = df_vide.copy()
    
    for puuid in account_IDs_list:
        time.sleep(1.5)
        match_ID_puller(matchID_list, puuid, startTime=mois[1][0], endTime=mois[1][1])
    
    df_travail[mois[0]] = matchID_list
    df1 = df1.combine_first(df_travail)

In [50]:
df1.to_csv('MatchId.csv', mode='x')

**Étape 4 : Mise en place de la base de données"**

In [81]:
import requests
import time
import pandas as pd
import numpy as np

region1 = "europe"
matchIDs = pd.read_csv("MatchId.csv")

In [82]:
def match_data_collector (gameID):
    match_url = "https://{}.api.riotgames.com/lol/match/v5/matches/{}?api_key={}".format(region1, gameID ,api_key)
    match_api_data = requests.get(match_url).json()
    match_data = np.empty(10, dtype = object)
    for i in range(len(match_api_data['info']['participants'])):
        participant_info = match_api_data['info']['participants'][i]
        summonerName = participant_info["summonerName"]
        teamPosition = participant_info["teamPosition"]
        teamId = participant_info["teamId"]
        championName = participant_info["championName"]
        if participant_info["win"] == True :
            outcome = 'Victoire'
        else :
            outcome = 'Défaite'
        if teamId == 100 :
            if teamPosition == 'TOP':
                match_data[0] = [summonerName,championName,'Blue Top', outcome]
            elif teamPosition == 'JUNGLE':
                match_data[1] = [summonerName,championName,'Blue Jungle', outcome]
            elif teamPosition == 'MIDDLE':
                match_data[2] = [summonerName,championName,'Blue Mid', outcome]
            elif teamPosition == 'BOTTOM':
                match_data[3] = [summonerName,championName,'Blue Carry', outcome]
            elif teamPosition == 'UTILITY':
                match_data[4] = [summonerName,championName,'Blue Support', outcome]
            else :
                print( teamPosition , summonerName , championName , teamId , "not found" )
        elif teamId == 200 :
            if teamPosition == 'TOP':
                match_data[5] = [summonerName,championName,'Red Top', outcome]
            elif teamPosition == 'JUNGLE':
                match_data[6] = [summonerName,championName,'Red Jungle', outcome]
            elif teamPosition == 'MIDDLE':
                match_data[7] = [summonerName,championName,'Red Mid', outcome]
            elif teamPosition == 'BOTTOM':
                match_data[8] = [summonerName,championName,'Red Carry', outcome]
            elif teamPosition == 'UTILITY':
                match_data[9] = [summonerName,championName,'Red Support', outcome]
            else :
                print( teamPosition , summonerName , championName , teamId , "not found" )
        else :
            print ( teamPosition , summonerName , championName , teamId ,"team not found" )
    return match_data

Ici, j'ai tout d'abord essayer de construire un DataFrame plus "lisible" avec les multi-indices, cependant mon binôme m'a plus tard dit que c'était peu pratique à utiliser pour la partie de scraping, j'ai donc changé le format plus tard.

In [133]:
multi_indices = []

for i in range(matchIDs.shape[0]) :
    multi_indices.append(("Partie " + str(i) , 'Pseudo '))
    multi_indices.append(("Partie " + str(i) , 'Champion'))
    multi_indices.append(("Partie " + str(i), 'Résultat'))

multi_indices = pd.MultiIndex.from_tuples(multi_indices, names = ("Partie", "Infos"))
positions = ["Blue Top", "Blue Jungle", "Blue Mid", "Blue Carry", "Blue Support", "Red Top", "Red Jungle", "Red Mid", "Red Carry", "Red Support"]

games_data_oct = np.empty([3*matchIDs.shape[0],10], dtype = object)
df_oct = pd.DataFrame(games_data, index = multi_indices, columns = positions)

In [17]:
for i in range(0, 100) : 
        #range(0, len(matchIDs["Octobre"]))
    try :
        infos = (match_data_collector(matchIDs["Octobre"][i]))
        if any(elem is None for elem in infos) == False :
            for j in range (len(infos)):
                df_oct[infos[j][2]]["Partie " + str(i)] = [infos[j][0], infos[j][1], infos[j][3]]
        time.sleep(1.5)
    except KeyError :
        print("keyerror")
        break

 GárPeR Poppy 200 not found
 Paddddy Lillia 100 not found
 Maranc Blitzcrank 100 not found
 SAKEN JUNIOR Pyke 200 not found
 H0rny Potter Twitch 200 not found
 tempaadjidcvijyp Sett 100 not found
 Zíma Sejuani 100 not found
 TheMilkywayLord Jayce 100 not found
 Musse G Ekko 100 not found
 Urpog Urgot 200 not found
 Trahirand Vayne 200 not found
 Xin always mid Twitch 200 not found
 VoooBoooBs LeeSin 200 not found
 NonimoEUW Leblanc 200 not found
 chunyin111 Soraka 100 not found
 Allanokoji MissFortune 200 not found
 m1co Viego 200 not found
 ceminem Fizz 100 not found
 TFT INTERS MissFortune 100 not found
 LykoJ ovo Ekko 200 not found
 Pongo Pygmaeus Gragas 100 not found
 Hanma Zed Zed 100 not found
 J4 is the way JarvanIV 100 not found
 lovein798 Lulu 100 not found
 SL Jalleba Ashe 200 not found
 Bαcalhau Soraka 200 not found


In [162]:
index_with_nan = df_oct.index[df_oct.isnull().any(axis = 1)]
df_oct = df_oct.drop(index_with_nan)
df_oct

Blue Top Blue Jungle        Blue Mid    Blue Carry  \
Partie      Infos                                                            
Partie 0    Pseudo     TripleBen  Dark Memer        CroKette       ŃIRVANA   
            Champion       Nasus         Zac             Lux         Kaisa   
            Résultat     Défaite     Défaite         Défaite       Défaite   
Partie 1    Pseudo        but ok     Kanuera   Meep Enjooyer         tjfgx   
            Champion       Viego      Khazix          Viktor        Twitch   
...                          ...         ...             ...           ...   
Partie 4137 Champion  Tryndamere       Shaco             Vex        KogMaw   
            Résultat    Victoire    Victoire        Victoire      Victoire   
Partie 4138 Pseudo     adiShow11        Sore  kos o5t Israel  hi im ashrye   
            Champion     Camille       Shaco            Fizz         Vayne   
            Résultat    Victoire    Victoire        Victoire      Victoire   

                          Blue Support           Red Top   Red Jungle  \
Partie      Infos                                                       
Partie 0    Pseudo            Bioniks1            monkaS      N0EQUAL   
            Champion            Soraka         Gangplank       Rengar   
            Résultat           Défaite          Victoire     Victoire   
Partie 1    Pseudo             Vaati73            Lewite    raymmondd   
            Champion             Yuumi             Vayne      Nidalee   
...                                ...               ...          ...   
Partie 4137 Champion             Janna              Shen     Pantheon   
            Résultat          Victoire           Défaite      Défaite   
Partie 4138 Pseudo    iguessjustbetter  LeSko ROARRRRRRR  Yllätykseen   
            Champion              Lulu              Sion   Blitzcrank   
            Résultat          Victoire           Défaite      Défaite   

                           Red Mid  Red Carry    Red Support  
Partie      Infos                                             
Partie 0    Pseudo      CG Kinneke     Bendåx        Jonperi  
            Champion         Yasuo       Ashe         Maokai  
            Résultat      Victoire   Victoire       Victoire  
Partie 1    Pseudo         tuctuc1  qfxnhfzkl          Bingu  
            Champion         Akali     Ezreal         Thresh  
...                            ...        ...            ...  
Partie 4137 Champion         Akali      Kaisa         Velkoz  
            Résultat       Défaite    Défaite        Défaite  
Partie 4138 Pseudo    BendingSchme      Υυumi  Time of Dying  
            Champion        Qiyana       Ashe           Sona  
            Résultat       Défaite    Défaite        Défaite  

[12339 rows x 10 columns]

In [141]:
df_oct

Blue Top Blue Jungle        Blue Mid    Blue Carry  \
Partie      Infos                                                            
Partie 0    Pseudo     TripleBen  Dark Memer        CroKette       ŃIRVANA   
            Champion       Nasus         Zac             Lux         Kaisa   
            Résultat     Défaite     Défaite         Défaite       Défaite   
Partie 1    Pseudo        but ok     Kanuera   Meep Enjooyer         tjfgx   
            Champion       Viego      Khazix          Viktor        Twitch   
...                          ...         ...             ...           ...   
Partie 4137 Champion  Tryndamere       Shaco             Vex        KogMaw   
            Résultat    Victoire    Victoire        Victoire      Victoire   
Partie 4138 Pseudo     adiShow11        Sore  kos o5t Israel  hi im ashrye   
            Champion     Camille       Shaco            Fizz         Vayne   
            Résultat    Victoire    Victoire        Victoire      Victoire   

                          Blue Support           Red Top   Red Jungle  \
Partie      Infos                                                       
Partie 0    Pseudo            Bioniks1            monkaS      N0EQUAL   
            Champion            Soraka         Gangplank       Rengar   
            Résultat           Défaite          Victoire     Victoire   
Partie 1    Pseudo             Vaati73            Lewite    raymmondd   
            Champion             Yuumi             Vayne      Nidalee   
...                                ...               ...          ...   
Partie 4137 Champion             Janna              Shen     Pantheon   
            Résultat          Victoire           Défaite      Défaite   
Partie 4138 Pseudo    iguessjustbetter  LeSko ROARRRRRRR  Yllätykseen   
            Champion              Lulu              Sion   Blitzcrank   
            Résultat          Victoire           Défaite      Défaite   

                           Red Mid  Red Carry    Red Support  
Partie      Infos                                             
Partie 0    Pseudo      CG Kinneke     Bendåx        Jonperi  
            Champion         Yasuo       Ashe         Maokai  
            Résultat      Victoire   Victoire       Victoire  
Partie 1    Pseudo         tuctuc1  qfxnhfzkl          Bingu  
            Champion         Akali     Ezreal         Thresh  
...                            ...        ...            ...  
Partie 4137 Champion         Akali      Kaisa         Velkoz  
            Résultat       Défaite    Défaite        Défaite  
Partie 4138 Pseudo    BendingSchme      Υυumi  Time of Dying  
            Champion        Qiyana       Ashe           Sona  
            Résultat       Défaite    Défaite        Défaite  

[12417 rows x 10 columns]

On change le format du DataFrame.

In [159]:
positions = ["Blue Top", "Blue Jungle", "Blue Mid", "Blue Carry", "Blue Support", "Red Top", "Red Jungle", "Red Mid", "Red Carry", "Red Support", "Outcome"]
games_data_oct = np.empty([df_oct.shape[0]//3,11], dtype = object)

for i in range(games_data_oct.shape[0]) :
    for j in range (10) :
        games_data_oct[i][j] = [df_oct.iloc[i*3][j], df_oct.iloc[i*3+1][j]]
    if df.iloc[i*3+2][0] == 'Victoire':
        games_data_oct[i][10] = 'Victoire des Bleus'
    else :
        games_data_oct[i][10] = 'Victoire des Rouges'

df_final_oct

,Blue Top,Blue Jungle,Blue Mid,Blue Carry,Blue Support,Red Top,Red Jungle,Red Mid,Red Carry,Red Support,Outcome
0,"[TripleBen, Nasus]","[Dark Memer, Zac]","[CroKette, Lux]","[ŃIRVANA, Kaisa]","[Bioniks1, Soraka]","[monkaS, Gangplank]","[N0EQUAL, Rengar]","[CG Kinneke, Yasuo]","[Bendåx, Ashe]","[Jonperi, Maokai]",Victoire des Rouges
1,"[but ok, Viego]","[Kanuera, Khazix]","[Meep Enjooyer, Viktor]","[tjfgx, Twitch]","[Vaati73, Yuumi]","[Lewite, Vayne]","[raymmondd, Nidalee]","[tuctuc1, Akali]","[qfxnhfzkl, Ezreal]","[Bingu, Thresh]",Victoire des Bleus
2,"[Lewite, Vayne]","[Lundqvistt, Volibear]","[Chieurdepatate, Swain]","[qfxnhfzkl, Twitch]","[spray em 50, Amumu]","[Sk8 The Infinity, Sejuani]","[ChemtrailsOverUs, Singed]","[Sekoiya, Yasuo]","[juicy ladyboy, Jinx]","[lοve too hard, Lulu]",Victoire des Bleus
3,"[PandaDogManPig, Aatrox]","[Tukki8, Zac]","[ASTROFISTER3333, Orianna]","[Timidt, Ashe]","[CLHB, Nautilus]","[Zegabon, Jayce]","[Age 3O Mechanics, Ekko]","[Lewite, Yasuo]","[qfxnhfzkl, Twitch]","[Elíte, Nami]",Victoire des Bleus
4,"[OverSharkt, Jayce]","[Nono144, Pantheon]","[FreeParking, Syndra]","[the høød, Draven]","[Hihi Haha Miau, Nami]","[lamygale4, DrMundo]","[FlexRankMatters, Nunu]","[MiriaelSabathiel, Xerath]","[Jokke, Twitch]","[NoIngameChatDiff, Janna]",Victoire des Rouges
...,...,...,...,...,...,...,...,...,...,...,...
4134,"[Ianv, Sejuani]","[Malwarwick, Warwick]","[WR Thirteen, Syndra]","[Paxii, Tristana]","[Kongalla, Yuumi]","[BrunetteMaleńka, Kaisa]","[Sore, Shaco]","[Be Cool Everyone, Vex]","[QinZKKnox, Kalista]","[vanonX, Thresh]",Victoire des Rouges
4135,"[Erik Killmonger, Kayle]","[xqxpvaeke, LeeSin]","[letliveagain, Fizz]","[MrG3nius, Samira]","[SimpForPumpkin, Alistar]","[KezzÂ, Fiora]","[Sore, Shaco]","[Vaneldeoriol, Diana]","[MelanchoIy, Xayah]","[Secret Tactics, Lulu]",Victoire des Bleus
4136,"[NEED MUCH DOPE, Maokai]","[Sore, Shaco]","[Chozer, Syndra]","[SAKATA GINTOKl, Tristana]","[Extrem Tobi, Blitzcrank]","[Ørne, Ornn]","[Hiroto, Rengar]","[Team Prepa Segpa, Zed]","[Fandar, Ashe]","[Companiion, Thresh]",Victoire des Rouges
4137,"[Top Ravellino, Tryndamere]","[Sore, Shaco]","[ Δscention, Vex]","[Hungry Cannibal, KogMaw]","[tempujsiiscchthp, Janna]","[Delirium Red, Shen]","[Hellne, Pantheon]","[MrDMG, Akali]","[EL Chicksen, Kaisa]","[Hober Mallow, Velkoz]",Victoire des Bleus


In [ ]:
df_final_oct.to_csv('MatchsOctobre.csv', mode='x')

On Peut faire la même chose pour le mois de Novembre

In [53]:
positions = ["Blue Top", "Blue Jungle", "Blue Mid", "Blue Carry", "Blue Support", "Red Top", "Red Jungle", "Red Mid", "Red Carry", "Red Support", "Outcome"]
games_data_nov = np.empty([matchIDs.shape[0],11], dtype = object)

In [79]:
for i in range(0, 100) :
        #range(0, len(matchIDs["Novembre"]))
    try :
        time.sleep(1.5)
        infos = (match_data_collector(matchIDs["Novembre"][i]))
        if any(elem is None for elem in infos) == False :
            for j in range (len(infos)):
                games_data_nov[i][j] = [infos[j][0], infos[j][1]]
            if infos[0][3] == 'Victoire':
                games_data_nov[i][10] = 'Victoire des Bleus'
            else :
                games_data_nov[i][10] = 'Victoire des Rouges'
    except KeyError :
        print("keyerror")
        break

keyerror


In [156]:
df_final_nov = pd.DataFrame(games_data_nov,  columns = positions)
index_with_nan_nov = df_final_nov.index[df_final_nov.isnull().any(axis = 1)]
df_final_nov = df_final_nov.drop(index_with_nan_nov)
df_final_nov

0

In [ ]:
df_final_nov.to_csv('MatchsNovembre.csv', mode='x')

0                 ['aoi hana', 'Fiora']
1                 ['aoi hana', 'Fiora']
2       ['RedyycsOnTwitter', 'Sejuani']
3            ['ArmandhI', 'Tryndamere']
4                ['LOKINS', 'Renekton']
                     ...               
3560          ['Berisleasimb', 'Neeko']
3561         ['Sϊn Of Greed', 'KSante']
3562               ['Aidamák', 'Yasuo']
3563                  ['Za cao', 'Zac']
3564                  ['4harp', 'Gwen']
Name: Blue Top, Length: 3565, dtype: object